<a href="https://colab.research.google.com/github/ashishpoudel995/Word-Segmentation-for-Nepali-Language/blob/main/Word_Segmentation_for_Nepali_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word Segmentation for Nepali Language**

## **1. Storing the most common non verb Words**
##### *3000 common words taken from website(3000mostcommonwords.com) and just storing non verb words*
##### *Verbal Inflections implemented in later stages. So, no need to store Verbs as of now*

In [ ]:
import pandas as pd

In [ ]:
csv=pd.read_csv("3000-common-words.csv")

In [ ]:
print(csv.head())

   Sr    Words                P.O.S Level Nepali Word
0   1    a, an  indefinite, article    A1      एक, एक
1   2  abandon                 Verb    B2     त्याग्न
2   3  ability                 Noun    A2      क्षमता
3   4     able            Adjective    A2       सक्षम
4   5    about         prep. Adverb    A1      बारेमा


In [ ]:
words_collection=[]
for index, row in csv.iterrows():
  if row["P.O.S"]!="Verb":
    if row["Nepali Word"] not in words_collection:
      words_collection.append(row["Nepali Word"])

In [ ]:
print(len(words_collection))

2204


##**Index:**
#####LGH-Low grade Honorific
#####MGH-Mid grade Honorific
#####HGH-High grade Honorific

##**2. Regular Verbs- Type 1**

In [ ]:
#Verb Stems
verb_stems=["घुम्","सुत्","पढ्","पर्","गर्","हेर्","सुन्","भन्","बोल्","चल्","मुत्","लेख्","बुझ्","उठ्","खेल्","सोच्","खोज्","हास्","हाक्","भेट्","बेच्","किन्","देख्","थुक्","चोर्","जीत्","खोल्","पोल्","काट्","सक्","सिक्","रोक्","थाप्","पुग्","पिट्","बग्","चल्","मान्","माग्","नाच्","बाच्","फेर्","मार्","खिच्","थिच्","फुल्","बाल्","निभ्","कुद्","तात्","हार्","राख्","मिल्","खस्"]

conjugated_forms=[]

##All possible conjugations for Regular Verbs- Type 1

for verb in verb_stems:
  #Affirmative Paradigm
  #1s
  conjugated_forms.append(verb+"छु")
  conjugated_forms.append(verb[0:len(verb)-1]+"े")
  conjugated_forms.append(verb[0:len(verb)-1]+"ाैला")

  #1p
  conjugated_forms.append(verb+"छौ")
  conjugated_forms.append(verb+"यौ")
  conjugated_forms.append(verb[0:len(verb)-1]+"ाैला")

  #2,s,m,LGH
  conjugated_forms.append(verb+"छस्")
  conjugated_forms.append(verb[0:len(verb)-1]+"िस्")
  conjugated_forms.append(verb+"लास्")

  #2,s,f,LGH
  conjugated_forms.append(verb+"छेस्")
  conjugated_forms.append(verb[0:len(verb)-1]+"िस्")
  conjugated_forms.append(verb+"लिस्")

  #2,s,m,MGH
  conjugated_forms.append(verb+"छौ")
  conjugated_forms.append(verb+"यौ")
  conjugated_forms.append(verb[0:len(verb)-1]+"ाैला")

  #2,s,f,MGH
  conjugated_forms.append(verb+"छेउ")
  conjugated_forms.append(verb+"यौ")
  conjugated_forms.append(verb[0:len(verb)-1]+"ाैली")

  #2,p,m/f,MGH
  conjugated_forms.append(verb+"छौ")
  conjugated_forms.append(verb+"यौ")
  conjugated_forms.append(verb[0:len(verb)-1]+"ाैला")

  #3,s,m,LGH
  conjugated_forms.append(verb+"छ")
  conjugated_forms.append(verb+"यो")
  conjugated_forms.append(verb+"ला")

  #3,s,f,LGH
  conjugated_forms.append(verb+"छे")
  conjugated_forms.append(verb[0:len(verb)-1]+"ी")
  conjugated_forms.append(verb+"ली")

  #3,s,m,MGH
  conjugated_forms.append(verb+"छन्")
  conjugated_forms.append(verb[0:len(verb)-1]+"े")
  conjugated_forms.append(verb+"लान्")

  #3,s,f,MGH
  conjugated_forms.append(verb+"छिन्")
  conjugated_forms.append(verb+"िन्")
  conjugated_forms.append(verb+"लिन्")

  #3,p,m/f,MGH
  conjugated_forms.append(verb+"छन्")
  conjugated_forms.append(verb[0:len(verb)-1]+"े")
  conjugated_forms.append(verb+"लान्")

  #HGH
  conjugated_forms.append(verb+"नुहुन्छ")
  conjugated_forms.append(verb+"नुभयो")
  conjugated_forms.append(verb+"नुहोला")

  #Negative Paradigm
  #1s
  conjugated_forms.append(verb+"दिन")
  conjugated_forms.append(verb+"िन")
  conjugated_forms.append("न"+verb[0:len(verb)-1]+"ुला")
  #1p
  conjugated_forms.append(verb+"दैनौ")
  conjugated_forms.append(verb+"िनौ")
  conjugated_forms.append("न"+verb[0:len(verb)-1]+"ाैला")

  #2,s,m,LGH
  conjugated_forms.append(verb+"दैनस्")
  conjugated_forms.append(verb[0:len(verb)-1]+"िनस्")
  conjugated_forms.append("न"+verb+"लास्")

  #2,s,f,LGH
  conjugated_forms.append(verb+"दैनेस्")
  conjugated_forms.append(verb[0:len(verb)-1]+"िनेस्")
  conjugated_forms.append("न"+verb+"लिस्")

  #2,s,m,MGH
  conjugated_forms.append(verb+"दैनौ")
  conjugated_forms.append(verb[0:len(verb)-1]+"ेनौ")
  conjugated_forms.append("न"+verb[0:len(verb)-1]+"ाैला")

  #2,s,f,MGH
  conjugated_forms.append(verb+"दिनौ")
  conjugated_forms.append(verb[0:len(verb)-1]+"िनौ")
  conjugated_forms.append("न"+verb[0:len(verb)-1]+"ाैली")

  #2,p,m/f,MGH
  conjugated_forms.append(verb+"दैनौ")
  conjugated_forms.append(verb[0:len(verb)-1]+"ेनौ")
  conjugated_forms.append("न"+verb[0:len(verb)-1]+"ाैला")

  #3,s,m,LGH
  conjugated_forms.append(verb+"दैन")
  conjugated_forms.append(verb[0:len(verb)-1]+"ेन")
  conjugated_forms.append("न"+verb+"ला")

  #3,s,f,LGH
  conjugated_forms.append(verb+"दैने")
  conjugated_forms.append(verb[0:len(verb)-1]+"िन")
  conjugated_forms.append("न"+verb+"ली")

  #3,s,m,MGH
  conjugated_forms.append(verb+"दैनन्")
  conjugated_forms.append(verb[0:len(verb)-1]+"ेनन्")
  conjugated_forms.append("न"+verb+"लान्")

  #3,s,f,MGH
  conjugated_forms.append(verb+"दैनिन्")
  conjugated_forms.append(verb[0:len(verb)-1]+"िनन्")
  conjugated_forms.append("न"+verb+"लिन्")

  #3,p,m/f,MGH
  conjugated_forms.append(verb+"दैनन्")
  conjugated_forms.append(verb[0:len(verb)-1]+"ेनन्")
  conjugated_forms.append("न"+verb+"लान्")

  #HGH
  conjugated_forms.append(verb+"नुहुदैन")
  conjugated_forms.append(verb+"नुभएन")
  conjugated_forms.append("न"+verb+"नुहोला")

  #Past Habitual Paradigm
  #1s(m/f)
  conjugated_forms.append(verb+"थे")
  conjugated_forms.append(verb+"दिनथे")
  conjugated_forms.append(verb+"दैनथे")

  #1p
  conjugated_forms.append(verb+"थेउ")
  conjugated_forms.append(verb+"दैनथेउ")

  #2,s,m,LGH
  conjugated_forms.append(verb+"थिस्")
  conjugated_forms.append(verb+"दैनथिस्")

  #2,s,f,LGH
  conjugated_forms.append(verb+"थिस्")
  conjugated_forms.append(verb+"दैनथिस्")

  #2,s,m,MGH
  conjugated_forms.append(verb+"थेउ")
  conjugated_forms.append(verb+"दैनथेउ")

  #2,s,f,MGH
  conjugated_forms.append(verb+"थेउ")
  conjugated_forms.append(verb+"दैनथेउ")

  #2,p,m/f,MGH
  conjugated_forms.append(verb+"थेउ")
  conjugated_forms.append(verb+"दैनथेउ")

  #3,s,m,LGH
  conjugated_forms.append(verb+"थ्यो")
  conjugated_forms.append(verb+"दैनथ्यो")

  #3,s,f,LGH
  conjugated_forms.append(verb+"थी")
  conjugated_forms.append(verb+"दैनथी")

  #3,s,m,MGH
  conjugated_forms.append(verb+"थे")
  conjugated_forms.append(verb+"दैनथे")

  #3,s,f,MGH
  conjugated_forms.append(verb+"थिन्")
  conjugated_forms.append(verb+"दैनथिन्")

  #3,p,m/f,MGH
  conjugated_forms.append(verb+"थे")
  conjugated_forms.append(verb+"दैनथे")

  #HGH
  conjugated_forms.append(verb+"नुहुन्थ्यो")
  conjugated_forms.append(verb+"नुहुदैनथ्यो")
  conjugated_forms.append(verb+"नुहुन्नथ्यो")

In [ ]:
print("Total Verb stems are:",len(verb_stems))
print("Total conjugated Forms of the verbs are:",len(conjugated_forms))

Total Verb stems are: 54
Total conjugated Forms of the verbs are: 5724


In [ ]:
suffix_type1=["छु","छौ","यौ","छस्","लास्","छेस्","लिस्","छेउ","छ","यो","ला","छे","ली","छन्","लान्","छिन्","िन्","लिन्","नुहुन्छ","नुभयो","नुहोला","दिन","िन","दैनौ","िनौ","दैनस्","दैनेस्","दिनौ","दैन","दैने","दैनन्","दैनिन्","नुहुदैन","नुभएन","थे","दिनथे","दैनथे","थेउ","दैनथेउ","थिस्","दैनथिस्","थ्यो","दैनथ्यो","थी","दैनथी","थिन्","दैनथिन्","नुहुन्थ्यो","नुहुदैनथ्यो","नुहुन्नथ्यो"]
suffix_type2=["े","ाैला","िस्","ाैली","ी","िनस्","िनेस्","ेनौ","िनौ","ेन","िन","ेनन्","िनन्"]
suffix_type3=["लास्","लिस्","ला","ली","लान्","लिन्","नुहोला"]
suffix_type4=["ुला","ाैला","ाैली"]

def check_if_valid_verbConjugation(word):
  for i in range(len(word)):
    curr=word[0:i+1]
    rem=word[i+1:]
    if (curr in verb_stems and rem in suffix_type1):
      return True
    elif ((curr+chr(2381)) in verb_stems and rem in suffix_type2):
      return True
    elif ord(word[0])==2344:
      curr=word[1:i+1]
      rem=word[i+1:]
      if (curr in verb_stems and rem in suffix_type3):
        return True
      elif ((curr+chr(2381)) in verb_stems and rem in suffix_type4):
        return True
  return False

##**3. Simpler Matching Algorithm**

In [ ]:
input_string="मएकसक्षमडिजाइनरमघुम्छु"
res=""
start=0
for i in range (len(input_string)):
    #tracking the current word
    curr_string=input_string[start:i+1]

    #check if the current word is present in either words_collection(non verb collection) or generated conjugated forms of the verbs.
    if(curr_string in words_collection or check_if_valid_verbConjugation(curr_string)):
        #if the word is considered to be a valid word, append it to result and add space(Nepali is a segmented language)
        res=res+curr_string+" "
        start=i+1

#Finally print the result of the Simpler Matching Algorithm
print("The segmented words are:")
print(res)

The segmented words are:
म एक सक्षम डिजाइन र म घुम्छ 


## **4. Maximal Matching Algorithm**

In [ ]:
res=""
start=0
while start<len(input_string):
    max_word=""
    for i in range(start, len(input_string)):
        curr_word=input_string[start:i+1]
        if curr_word in words_collection or check_if_valid_verbConjugation(curr_word):
          if len(curr_word)>len(max_word):
            max_word=curr_word
    if(len(max_word)==0):
        res+=input_string[i:i+1]+" "
        start=start+1
    else:
        start=start+len(max_word)
        res+=max_word+" "
print(res)

म एक सक्षम डिजाइनर म घुम्छु 


## **5. Bidirectional Maximal Matching Algorithm**

In [ ]:
res_left=""

#Scan from left
start=0
left_count=0
while start<len(input_string):
    max_word=""
    for i in range(start, len(input_string)):
        curr_word=input_string[start:i+1]
        if curr_word in words_collection or check_if_valid_verbConjugation(curr_word):
          if len(curr_word)>len(max_word):
            max_word=curr_word
    if(len(max_word)==0):
        left_count=left_count+1
        res_left+=input_string[i:i+1]+" "
        start=start+1
    else:
        start=start+len(max_word)
        res_left+=max_word+" "

#Scan from right
res_right=""
start=len(input_string)-1
right_count=0
while start>=0:
    max_word=""
    i=start
    while i>=0:
        curr_word=input_string[i:start+1]
        if curr_word in words_collection or check_if_valid_verbConjugation(curr_word):
          if len(curr_word)>len(max_word):
            max_word=curr_word
        i=i-1
    if (len(max_word)==0):
        res_right+=input_string[i:i+1]+" "
        start=start-1
        right_count=right_count+1
    else:
        start=start-len(max_word)
        res_right+=max_word+" "

#Print the best scan
if left_count<=right_count:
  print(res_left)
else:
  print(res_right)

म एक सक्षम डिजाइनर म घुम्छु 


## **Maximum Matching Algorithm**

## **Prac**